In [1]:
import  numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb 
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval


randomseed=42
np.random.seed(randomseed)

In [2]:

xtest=np.array(pd.read_csv('../dataset/xtest.txt'))
xtrain=np.array(pd.read_csv('../dataset/xtrain.txt'))
ytest_original=np.array(pd.read_csv('../dataset/ytest.txt')).ravel()
ytrain_original =np.array(pd.read_csv('../dataset/ytrain.txt')).ravel()

# data=datasets.load_wine()
# x=data.data
# y=data.target

# data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))

# data = pd.read_csv("../dataset/ionosphere.data",  header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))




# xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 

ytrain=ytrain_original.copy()
ytest=ytest_original.copy() 

# ytrain, ytest = swapcolumns(ytrain, ytest, 2)

In [3]:


clf=[]
acc=[]
finalacc=[]
ypredproba_all=[]
ypredconfprob_all=[]


#================================================= 

# Class 2
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,2)
#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))


xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)
xgbpred=xgbc.predict(xtest)
print('original score 2',m.f1_score(ytest,xgbpred,average='weighted'))


#================================================= 

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)


#=================================================
svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
svc.fit(xtrain,ytrain)

svcpred=svc.predict(xtest)
print(m.f1_score(ytest,svcpred,average='weighted'))

clf.append(svc)
acc.append(m.f1_score(ytest,svcpred,average='weighted'))
ypredproba_all.append(1-svc.predict_proba(xtest)[:,1])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)

xgbpred=xgbc.predict(xtest)
print(m.f1_score(ytest,xgbpred,average='weighted'))


clf.append(xgbc)
acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
ypredproba_all.append(xgbc.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
#=================================================
# generate combinations of features 12,6
comb = list(itertools.combinations(np.arange(0, 12, 1), 7))

# generate 50 random numbers
randnums = []
for i in range(50):
    randnums.append(random.randrange(0, len(comb)))

print(randnums)

comb = np.array(comb)[randnums, :]


original score 0.8886190707463169
original score 2 0.9114979197447021
0.8886190707463169
0.9178477001592273
0.9114979197447021
[619, 33, 167, 241, 271, 351, 482, 475, 779, 516, 4, 560, 525, 455, 499, 438, 7, 55, 207, 241, 627, 611, 783, 44, 320, 606, 150, 186, 175, 181, 34, 224, 130, 121, 127, 313, 732, 63, 691, 496, 78, 346, 379, 89, 156, 289, 282, 761, 770, 638]


In [4]:
for i in range(len(comb)):
    print(i, " ==================== ", comb[i])

    rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
    rf.fit(xtrain[:, comb[i]], ytrain)
    rfpred = rf.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]])[:,0])

    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=1)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)

    
# #=================================================


0  ====================  [ 1  3  4  7  8  9 11]
0.8381496771461839
1  ====================  [ 0  1  2  3  5  9 10]
0.8867589442471787
2  ====================  [ 0  1  3  5  6  8 11]
0.8738805054594528
3  ====================  [ 0  1  5  7  8  9 10]
0.8943858956735599
4  ====================  [0 2 3 4 6 8 9]
0.8722700949024386
5  ====================  [ 0  2  4  6  9 10 11]
0.8653230433253751
6  ====================  [ 1  2  3  4  6  8 10]
0.8627520970908442
7  ====================  [ 1  2  3  4  5  9 11]
0.8452639770518461
8  ====================  [ 3  5  6  7  8 10 11]
0.5586426359575896
9  ====================  [ 1  2  3  5  9 10 11]
0.8504139628552249
10  ====================  [ 0  1  2  3  4  5 10]
0.8725485210611068
11  ====================  [ 1  2  4  6  8 10 11]
0.8643553587249411
12  ====================  [ 1  2  3  6  8 10 11]
0.8405141302537339
13  ====================  [ 0  5  6  7  8  9 10]
0.8440582606925398
14  ====================  [ 1  2  3  5  6  7 10]
0.86050671498001

In [5]:

for i in range(len(comb)):
    print(i, " ==================== ", comb[i])


    xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
    xgbmodel.fit(xtrain[:, comb[i]], ytrain)
    xgbmodelpred = xgbmodel.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, xgbmodelpred,average='weighted'))

    clf.append(xgbmodel)
    acc.append(m.f1_score(ytest, xgbmodelpred,average='weighted'))
    ypredproba_all.append(xgbmodel.predict_proba(xtest[:, comb[i]])[:,0])

    confmat = m.confusion_matrix(ytest, xgbmodelpred)
    confsumh = np.sum(confmat, axis=1)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)
# #=================================================

0  ====================  [ 1  3  4  7  8  9 11]
0.8828792516364157
1  ====================  [ 0  1  2  3  5  9 10]
0.9150527935480448
2  ====================  [ 0  1  3  5  6  8 11]
0.911721530549929
3  ====================  [ 0  1  5  7  8  9 10]
0.911721530549929
4  ====================  [0 2 3 4 6 8 9]
0.9040235837781851
5  ====================  [ 0  2  4  6  9 10 11]
0.8977287695859769
6  ====================  [ 1  2  3  4  6  8 10]
0.8825577714466604
7  ====================  [ 1  2  3  4  5  9 11]
0.8793695696100223
8  ====================  [ 3  5  6  7  8 10 11]
0.660011600573923
9  ====================  [ 1  2  3  5  9 10 11]
0.8905532095716144
10  ====================  [ 0  1  2  3  4  5 10]
0.913440317210938
11  ====================  [ 1  2  4  6  8 10 11]
0.8827204148986326
12  ====================  [ 1  2  3  6  8 10 11]
0.8878231504866767
13  ====================  [ 0  5  6  7  8  9 10]
0.8974385867889686
14  ====================  [ 1  2  3  5  6  7 10]
0.8905532095716144
1

In [6]:

# for i in range(len(comb)):
#     print(i, " ==================== ", comb[i])

#     svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
#     svc.fit(xtrain[:, comb[i]], ytrain)
#     svcpred=svc.predict(xtest[:, comb[i]])
    
#     print(m.f1_score(ytest,svcpred,average='weighted'))

#     clf.append(svc)
    
#     acc.append(m.f1_score(ytest,svcpred,average='weighted'))
#     ypredproba_all.append(1-svc.predict_proba(xtest[:, comb[i]])[:,1])

#     confmat = m.confusion_matrix(ytest, svcpred)
#     confsumh = np.sum(confmat, axis=1)
#     propconfmat = confmat.copy()
#     for i in range(propconfmat.shape[0]):
#         propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
#     ypredconfprob_all.append(propconfmat.ravel() / 100)
    
# # #=================================================


In [7]:
# pd_clf=pd.DataFrame(list(clf))
pd_acc=pd.DataFrame(acc) 
pd_ypredproba_all=pd.DataFrame(np.array(ypredproba_all))
pd_ypredconfprob_all=pd.DataFrame(ypredconfprob_all)

In [8]:
classname='class2.txt'
pd_acc.to_csv('pd_acc_'+classname,sep=',',index=False) 
pd_ypredproba_all.to_csv('pd_ypredproba_all_'+classname,sep=',',index=False)
pd_ypredconfprob_all.to_csv('pd_ypredconfprob_all_'+classname,sep=',',index=False)

In [9]:
pd.DataFrame(ytrain_original).to_csv('ytrain_original.txt',sep=',',index=False)
pd.DataFrame(ytest).to_csv('ytest_class2.txt',sep=',',index=False)

In [10]:
# svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
# svc.fit(xtrain,ytrain)

# svcpred=svc.predict(xtest)
# print(m.f1_score(ytest,svcpred,average='weighted'))

# temppp=svc.predict_proba(xtest)

In [11]:
# m.f1_score(ytest, np.argmax(temppp, axis=1),average='weighted')